# Daily retrain之訓練與測試

In [15]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import os
from sklearn.manifold import TSNE
import itertools
from dataPreprocess import read_dataset
from loss_functions import maximumMeanDiscrepancy
from sklearn.metrics import r2_score

In [2]:
targetSessionIndex = 16 # 選擇session

In [4]:
folderPath = './data'
fileList = sorted([i for i in os.listdir(folderPath) if i.endswith('.pkl.zip')])

In [5]:
targetSession = fileList[targetSessionIndex]

In [6]:
targetDataset = read_dataset(os.path.join(folderPath, targetSession), ['velocity_x', 'velocity_y'])

In [7]:
from model_tf import simpleDecodeModel

model = simpleDecodeModel()
model.compile(optimizer='adam', loss='mse')

In [8]:
TRAIN_COUNT = 5000

train_x = targetDataset['m1'][:TRAIN_COUNT, -5:, :]
train_y = targetDataset['movement'][:TRAIN_COUNT]
test_x = targetDataset['m1'][TRAIN_COUNT:, -5:, :]
test_y = targetDataset['movement'][TRAIN_COUNT:]

train_timestamp = targetDataset['timestamp'][:TRAIN_COUNT]
test_timestamp = targetDataset['timestamp'][TRAIN_COUNT:]

In [9]:
# z-score
train_y_mean, train_y_std = np.mean(train_y, axis=0), np.std(train_y, axis=0)
train_y = (train_y - train_y_mean) / train_y_std
test_y = (test_y - train_y_mean) / train_y_std

In [10]:
# train
model.fit(x=train_x, y=train_y, batch_size=128, epochs=100, shuffle=True, verbose=0)

In [11]:
# pred
pred = model.predict(x=test_x)

In [12]:
# 還原
pred = pred * train_y_std + train_y_mean
test_y = test_y * train_y_std + train_y_mean

In [16]:
# 計算R2
r2 = [r2_score(y_pred=pred[:, i], y_true=test_y[:, i]) for i in range(pred.shape[-1])]
print('vx:', r2[0])
print('vy:', r2[1])

vx: 0.7308415154542787
vy: 0.776087132511426


In [ ]:
# 輸出
result = pd.DataFrame(columns=[])
axis = ['x', 'y']
for n, a in enumerate(axis):
    result = result.append(pd.DataFrame({
        'timestamp': test_timestamp,
        'value': test_y[:, n],
        'type': 'groundtruth',   
        'movement': 'velocity',
        'axis': a,          
        'test_on': targetSessionIndex,
        'r2': r2[n]     
    }))
    result = result.append(pd.DataFrame({
        'timestamp': test_timestamp,
        'value': pred[:, n],
        'type': 'prediction',   
        'movement': 'velocity',
        'axis': a, 
        'test_on': targetSessionIndex,  
        'r2': r2[n]           
    }))
result.reset_index(inplace=True, drop=True)

result.to_csv('./results/trainOnTarget.csv', index=False, header=True)